## Решение проблемы с отсутствующими признаками

Большинство алгоритмов машинного обучения не могут работать с отсутствующими признакми; в нашем случае мы видели, что у атрибута `total_bedrooms` отсутствовали признаки для некоторого подмножества образцов. Можно исправить ситуацию несколькими способами:

1. избавиться от образцов, у которых отсутствуют признаки total_bedrooms:<br>
`housing.dropna(subset=["total_bedrooms"])`<br>

2. избавиться от самого атрибута total_bedrooms:<br>
`housing.drop("total_bedrooms", axis=1)`

3. заполнить отсутствующие признаки значениями (нулями, средним арифметическим, и тд.):<br>
`mean = housing["total_bedrooms"].mean()`<br>
`housing["total_bedrooms"].fillna(mean)`<br>
в последнем случае значение mean следует сохранить до конца вычислений, так как для заполненных данных пересчет этого значения может дать уже другое значение.

Окончание -`na` в этих методав является аббревиатурой *Not Available* (нет в наличии). Такие методы оказывают влияние только на те ячейки таблицы данных, в которых отсутствуют значения.

### Удаление образцов

Удалим образцы с отсутствующими значениями признака `total_bedrooms`

In [ ]:
cleared_housing = housing.dropna(subset=["total_bedrooms"])

In [ ]:
cleared_housing.info()

### Удаление признака

Удалим признак `total_bedrooms`:

In [ ]:
cleared_housing = housing.drop(["total_bedrooms", "ocean_proximity"], axis=1)

In [ ]:
cleared_housing.head()

### Заполнение отсутствующих ячеек

Получим копию исходной таблицы данных в переменную `filled_housing`. Вычислим среднее значение и медиану признака `total_bedrooms` при помощи методов `means()` и `median()`.

In [ ]:
filled_housing = housing.copy(deep=True)
mean = filled_housing["total_bedrooms"].mean()
median = filled_housing["total_bedrooms"].median()
mean, median

Заполненим отсутствующие ячейки признака `total_bedrooms` значением `mean` используя метод `fillna()`. Методь возвращает объект класса `Series`, который можно присвоить колонке в объекте `DataFrame`. Исходный объект, у которого вызывается метод при этом не меняется.

Если задать в методе параметр `inplace=True`, то произойдет заполнение значениями на месте, в колонке самого объекта. Метод при этом ничего не будет возвращать (`None`).

In [ ]:
type(filled_housing["total_bedrooms"].fillna(mean))

In [ ]:
# filled_housing["total_bedrooms"] = filled_housing["total_bedrooms"].fillna(mean)
filled_housing["total_bedrooms"].fillna(mean, inplace=True)

## Отделение целевого признака

Извлекаем данные целевого признака в виде одномерного массива NumPy и записываем их в переменную `y`. Остальные признаки извлекаем в виде двумерного массива NumPy и записываем в `X`. Для этого можно использовать метод `to_numpy()`.

In [ ]:
X = cleared_housing.drop(columns=['median_house_value']).to_numpy()
y = cleared_housing['median_house_value'].to_numpy()                # целевой признак

In [ ]:
print(X[:3])

In [ ]:
print(y[:3])

## Конвертация категориальных данных

Единственный текстовый атрибут в нашей выборке – это `ocean_proximity`.
Это не произвольные текстовые значения, возможные значения ограничены, поэтому здесь имеем дело с категоральным атрибутом. Большинство алгоритмов предпочитают работать с числами, поэтому целесообразно конвертировать эти категории из текстовой формы в числовую.

In [ ]:
housing_cat = housing[["ocean_proximity"]]
print(housing_cat.value_counts())

In [ ]:
housing_cat.head()

### [`OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)

Для конвертации категориальных значений в числовые, можно использовать класс `OrdinalEncoder` из модуля `preprocessing` библиотеки `sklearn`. Для этого создается экземпляр класса `OrdinalEncoder`. Вызывается метод `fit()`, который возвращает массив числовых значений, соответствующие категориям.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(dtype=np.float16)
ordinal_encoder.fit(housing_cat)
housing_cat_encoded = ordinal_encoder.transform(housing_cat)
type(housing_cat_encoded), housing_cat_encoded[:3]

Нзавания категорий хранятся в поле `categories_` объекта класса `OrdinalEncoder` в виде массива, а числовые значения, кодирующие категории, соответствуют индексам этих категорий.

In [ ]:
ordinal_encoder.categories_

Проблемой такого представления является то, что алгоритм может посчитать, что близкие значения параметра, например 0 и 1, свидетельствуют о том, что они ближе, чем, скажем, 0 и 4, что не соответствует действительности. Чтобы избежать таких проблем, можно создавать бинарный атрибут для категорий (*one-hot encoding*) при помощи класса `OneHotEncoder`.

### [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

Интерфейс подобен классу `OrdinalEncoder`, однако, метод `transorm()` возвращает разареженную матрицу (объект класса `csr_encoded`). Для того, чтобы получить матрицу NumPy, можно воспользоваться методом `toarray()`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
cat_encoder.fit(housing_cat)
housing_cat_encoded = cat_encoder.transform(housing_cat)
type(housing_cat_encoded)

In [ ]:
housing_cat_encoded.toarray()[:3]

In [ ]:
cat_encoder.categories_

Стоит иметь в виду, что если категорий очень много, то матрица NumPy может оказаться слишком расточительной, гораздо рациональнее будет использовать разреженную матрицу. Либо попробовать заменить признак не его упорядоченное представление. Например для `ocean_proximity` заменить на расстояние до океана.

In [ ]:
a